In [7]:
import open3d as o3d
import numpy as np
import copy

# Function to visualize the alignment
def draw_registration_result(source, target, transformation):
    """
    Visualizes source and target point clouds with the given transformation.
    """
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])  # Orange for source
    target_temp.paint_uniform_color([0, 0.651, 0.929])  # Blue for target
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries(
        [source_temp, target_temp],
        zoom=0.4459,
        front=[0.9288, -0.2951, -0.2242],
        lookat=[1.6784, 2.0612, 1.4451],
        up=[-0.3402, -0.9189, -0.1996]
    )


In [48]:
def icp(source, target, max_iterations=1000, tolerance=1e-5, voxel_size=0.08):
   
    # Downsampling to improve the speed
    source = source.voxel_down_sample(voxel_size)
    target = target.voxel_down_sample(voxel_size)

    # Convert to NumPy arrays
    source_points = np.asarray(source.points)
    target_points = np.asarray(target.points)

    # nearest neighbour search
    target_kd_tree = o3d.geometry.KDTreeFlann(target)

    # Initialize transformation matrix
    transformation = np.eye(4)

    for iteration in range(max_iterations):
        # finding the nearest neighbours
        matched_source_points = []
        for point in source_points:
            _, idx, _ = target_kd_tree.search_knn_vector_3d(point, 1)
            matched_source_points.append(target_points[idx[0]])
        matched_source_points = np.array(matched_source_points)

        # Computing the centroid
        source_centroid = np.mean(source_points, axis=0)
        target_centroid = np.mean(matched_source_points, axis=0)

        # computing the points
        centered_source = source_points - source_centroid
        centered_target = matched_source_points - target_centroid

        # computing R and T
        H = np.dot(centered_source.T, centered_target)  
        U, _, Vt = np.linalg.svd(H)
        R = np.dot(Vt.T, U.T)

        # verifying if R is valid
        if np.linalg.det(R) < 0:
            Vt[-1, :] *= -1
            R = np.dot(Vt.T, U.T)

        t = target_centroid - np.dot(R, source_centroid)  # Translation 

        # transformation matrix
        new_transformation = np.eye(4)
        new_transformation[:3, :3] = R
        new_transformation[:3, 3] = t

        
        source_points = (np.dot(R, source_points.T).T + t)
        transformation = np.dot(new_transformation, transformation)

       # checking convergence
        error = np.linalg.norm(centered_target - np.dot(R, centered_source.T).T)
        print(f"Iteration {iteration + 1}, Cost: {error:.6f}")

        if error < tolerance:
            print("Convergence reached!")
            break

    return transformation

In [55]:
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])


In [56]:
# running the algorithm
transformation = icp(source, target)

Iteration 1, Cost: 15.325495
Iteration 2, Cost: 12.249860
Iteration 3, Cost: 11.062279
Iteration 4, Cost: 10.430045
Iteration 5, Cost: 10.099111
Iteration 6, Cost: 9.967242
Iteration 7, Cost: 9.902031
Iteration 8, Cost: 9.859872
Iteration 9, Cost: 9.831996
Iteration 10, Cost: 9.813016
Iteration 11, Cost: 9.801291
Iteration 12, Cost: 9.793577
Iteration 13, Cost: 9.787373
Iteration 14, Cost: 9.783097
Iteration 15, Cost: 9.781128
Iteration 16, Cost: 9.779472
Iteration 17, Cost: 9.777069
Iteration 18, Cost: 9.773784
Iteration 19, Cost: 9.770826
Iteration 20, Cost: 9.767823
Iteration 21, Cost: 9.764632
Iteration 22, Cost: 9.761499
Iteration 23, Cost: 9.759949
Iteration 24, Cost: 9.759060
Iteration 25, Cost: 9.758474
Iteration 26, Cost: 9.758061
Iteration 27, Cost: 9.757498
Iteration 28, Cost: 9.756768
Iteration 29, Cost: 9.756398
Iteration 30, Cost: 9.756211
Iteration 31, Cost: 9.756122
Iteration 32, Cost: 9.756080
Iteration 33, Cost: 9.756048
Iteration 34, Cost: 9.756025
Iteration 35, Cost

In [57]:
print("Final Transformation Matrix:\n", transformation)
draw_registration_result(source, target, transformation)

Final Transformation Matrix:
 [[ 0.82588188  0.03295996 -0.562879    1.3108789 ]
 [-0.23759494  0.92567464 -0.29440635  1.1343654 ]
 [ 0.51133919  0.37688207  0.77232903 -1.41721981]
 [ 0.          0.          0.          1.        ]]


# Part b Kitti Dataset

In [53]:
source = o3d.io.read_point_cloud("kitti_frame1.pcd")
target = o3d.io.read_point_cloud("kitti_frame2.pcd")
transformation_kitti = icp(source, target)

Iteration 1, Cost: 179.348535
Iteration 2, Cost: 173.800717
Iteration 3, Cost: 171.834729
Iteration 4, Cost: 170.888464
Iteration 5, Cost: 170.280670
Iteration 6, Cost: 169.865565
Iteration 7, Cost: 169.608891
Iteration 8, Cost: 169.455068
Iteration 9, Cost: 169.315387
Iteration 10, Cost: 169.131891
Iteration 11, Cost: 168.847492
Iteration 12, Cost: 168.590179
Iteration 13, Cost: 168.402185
Iteration 14, Cost: 168.290435
Iteration 15, Cost: 168.203166
Iteration 16, Cost: 168.105815
Iteration 17, Cost: 167.984067
Iteration 18, Cost: 167.868344
Iteration 19, Cost: 167.778555
Iteration 20, Cost: 167.709655
Iteration 21, Cost: 167.657955
Iteration 22, Cost: 167.613368
Iteration 23, Cost: 167.573346
Iteration 24, Cost: 167.533051
Iteration 25, Cost: 167.492162
Iteration 26, Cost: 167.452525
Iteration 27, Cost: 167.404816
Iteration 28, Cost: 167.331812
Iteration 29, Cost: 167.234276
Iteration 30, Cost: 167.151612
Iteration 31, Cost: 167.090829
Iteration 32, Cost: 167.041546
Iteration 33, Cos

In [54]:
print("Final Transformation Matrix:\n", transformation_kitti)
draw_registration_result(source, target, transformation_kitti)

Final Transformation Matrix:
 [[ 9.99996548e-01 -2.50175463e-03 -8.03805016e-04 -8.62582038e+00]
 [ 2.49020663e-03  9.99898052e-01 -1.40600576e-02 -1.38944232e+00]
 [ 8.38897883e-04  1.40580074e-02  9.99900829e-01 -2.91630448e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
